### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [1]:
import pandas as pd
import numpy as np

In [96]:
ratings = pd.read_csv('ratings.csv')

In [97]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [98]:
ratings_new = ratings.groupby('userId').agg({'rating': 'count', 'timestamp': [max, min]})
ratings_new.head()

rating   timestamp            
        count         max         min
userId                               
1          20  1260759205  1260759108
2          76   835356246   835355395
3          51  1298932787  1298861589
4         204   949982274   949778714
5         100  1163375145  1163373044

In [99]:
ratings_new['lifetime'] = ratings_new['timestamp']['max'] - ratings_new['timestamp']['min']
ratings_new.head()

rating   timestamp             lifetime
        count         max         min         
userId                                        
1          20  1260759205  1260759108       97
2          76   835356246   835355395      851
3          51  1298932787  1298861589    71198
4         204   949982274   949778714   203560
5         100  1163375145  1163373044     2101

In [100]:
ratings_new.loc[ratings_new.rating['count'] > 100].head()

rating   timestamp               lifetime
        count         max         min           
userId                                          
4         204   949982274   949778714     203560
8         116  1154474527  1154389340      85187
15       1700  1469330735   997937239  471393496
17        363  1127476640  1127468587       8053
19        423   855195373   855190091       5282

__Комментарий к первому варианту решения (не получившемуся)__: вроде выделил дтатафрейм с пользователями где больше ста оценок и как будто понятно, как посчитать время жизни для конкретного пользователя (по айди), но попытки как-то посчитать это для всех функцией провалились — и я что-то в ступоре и не понимаю, почему все поломалось. Это если считать с помощью дейттайм (чего делать, видимо, не стоило, судя по тому решению, которое у меня выше все же вышло)

In [4]:
ratings_count = ratings.groupby('userId').count()
ratings_count

,movieId,rating,timestamp
userId,,,
1,20,20,20
2,76,76,76
3,51,51,51
4,204,204,204
5,100,100,100
...,...,...,...
667,68,68,68
668,20,20,20
669,37,37,37


In [5]:
ratings_count.loc[ratings_count.movieId > 100].head()

,movieId,rating,timestamp
userId,,,
4,204,204,204
8,116,116,116
15,1700,1700,1700
17,363,363,363
19,423,423,423


In [6]:
ratings_count.loc[ratings_count.movieId > 100].shape

(258, 3)

In [7]:
ratings_count100 = ratings_count[ratings_count.movieId > 100].reset_index()
ratings_count100.head()

,userId,movieId,rating,timestamp
0,4,204,204,204
1,8,116,116,116
2,15,1700,1700,1700
3,17,363,363,363
4,19,423,423,423


In [17]:
merged_ratings = ratings.merge(ratings_count100['userId'], on='userId', how='right')
merged_ratings.tail()

,userId,movieId,rating,timestamp
80482,671,6268,2.5,1065579370
80483,671,6269,4.0,1065149201
80484,671,6365,4.0,1070940363
80485,671,6385,2.5,1070979663
80486,671,6565,3.5,1074784724


In [13]:
len(merged_ratings.groupby('userId')) == len(ratings_count100)

True

In [15]:
(
    pd.to_datetime(merged_ratings.query('userId == 671').timestamp.max(), unit='s')
    - pd.to_datetime(merged_ratings.query('userId == 671').timestamp.min(), unit='s')
)

Timedelta('130 days 14:26:24')

In [59]:
for i in ratings_count100['userId']:
     
    f = (
        pd.to_datetime(merged_ratings[merged_ratings.userId == i].timestamp.max(), unit='s') 
        - pd.to_datetime(merged_ratings.query[merged_ratings.userId == i].timestamp.min(), unit='s')
    )

TypeError: 'method' object is not subscriptable

In [48]:
x = 19

In [49]:
(
    pd.to_datetime(merged_ratings[merged_ratings.userId == x].timestamp.max(), unit='s')
    - pd.to_datetime(merged_ratings[merged_ratings.userId == x].timestamp.min(), unit='s')
)

Timedelta('0 days 01:28:02')

In [60]:
def time_diff(ratings_count100):

    for i in ratings_count100['userId']:

            return  (
            pd.to_datetime(merged_ratings[merged_ratings.userId == i].timestamp.max(), unit='s')
            - pd.to_datetime(merged_ratings.query[merged_ratings.userId == i].timestamp.min(), unit='s')
            )

In [61]:
ratings_count100['diff'] = ratings_count100.apply(time_diff, axis = 1)

TypeError: 'numpy.int64' object is not iterable

### Задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

Необходимо сформировать две таблицы:

* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [63]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [64]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [65]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [66]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [74]:
no_address_df = (rzd
                 .merge(auto, how='outer', on='client_id')
                 .merge(air, how='outer', on='client_id')
                )
no_address_df

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [77]:
address_df = no_address_df.merge(client_base, how='outer', on='client_id')
address_df

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

__Попытка ответа__: Вопрос, есть в этих данных по местоположению еще и время, когда их запрпашивало приложение. Если есть, то можно попробовать это соотнести со временем покупке и взять для объединения те координаты, которые близки по времени ко времени визита или покупки (наверное, попадают в какой-то интервал, ну или прямо совпадают со временем визита). 

Если нет и это просто вообще список координат (по условию я не совсем понял), то даже не знаю — брать первую из них и пытаться как-то привязывать к визитам пользователя (но визитов же много из разных мест). Можно бы посмотреть, не совпадает ли вообще количество координат и визитов пользователя (но по условиям кажется, что нет)

Кажется, что можно как-то попробовать обобщать эти координаты (до основного города/страны пользовтеля) и уже их соотносить как-то с покупками/категориями, частотой визитов, что-нибудь такое

(очень сложно вот так абстрактно, конечно, рассуждать)